```
Copyright (c) MONAI Consortium
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Calling Vista-3D NIM on NVAIE

In this tutorial, we will cover the following:

- **Convert image to Nifty format:** The Vista-3D NIM requires files to be in compressed nifty (.nii.gz) format.  We use ITK to convert DICOM and other file formats to nifty.

- **Create a single-use URL to the image:** We use the website file.io to provide a single-use URL for our image.  The Vista-3D NIM requires a URL to the image to be processed.  File.io provides an API for receiving a file and returning a one-time use URL to that file.

- **Communicate with the Vista-3D NIM on NVAIE:** The Vista-3D NIM can be downloaded and run locally, but herein we are running it on the NVIDIA AI Enterprise (NVAIE) / NVIDIA GPU Cloud (NGC) servers.  You can get free NVAIE/NGC credits and an API Key for remotely accessing those servers by registering at https://build.nvidia.com/nvidia/vista-3d.  Also, you can apply for unlimited access as a researcher by joining the (free) NVIDIA Developer Program: https://developer.nvidia.com/join-nvidia-developer-program

- **Visualize and/or save the results:** We provide pyvista for visualization and use ITK to save the results in any of a variety of image formats.

Learn more about the Vista-3D foundation model for 3D CT segmentation:<br>
https://arxiv.org/abs/2304.02643

Learn more about NIMs:<br>
https://www.nvidia.com/en-us/ai/


## Obtaining an NVIDIA AI Enterprise / NVIDIA GPU Cloud (NVAIE/NGC) API Key

This demonstration runs the Vista-3D NIM (AI container) on the NVIDIA AI Enterprise servers, so an API key is required.  The good news is that any individual can get an API key and 1,000 to 5,000 free credits (processing one image requires one credit) when they first sign-up at:

- https://build.nvidia.com/nvidia/vista-3d

Also, you can apply for unlimited local access as a researcher by joining the (free) NVIDIA Developer Program:

- https://developer.nvidia.com/join-nvidia-developer-program

This API Key is specific to your account at NVIDIA.

We recommend storing and accessing your personal/private API key as an environment variable, rather than embedding it within your code where it may be accidentally shared with or viewed by others.  For example, in bash you can run this command in your environment or embed it into your ~/.bashrc file:

 `export NGC_API_KEY=<your key here>`

and then launch jupyter lab / vscode to run this notebook.

## Setup the python environment

- `itk`: used to support a variety of image file formats and to convert images for viewing via pyvista.
    - Learn more at https://github.com/InsightSoftwareConsortium/ITK
- `pyvista`: provides 2D and 3D scientific visualizations, compatible with VSCode and most jupyter notebook/lab systems
    - The `trame` option is used to provide interactive rendering support.
    - Learn more at https://docs.pyvista.org/ 

In [1]:
!python -c "import itk" || pip install -q "itk"
!python -c "import pyvista" || ipywidgets 'pyvista[all,trame]'

## Setup the imports

In [2]:
import io
import os
import requests
import tempfile
import zipfile

import itk
import pyvista as pv


## Upload a file to file.io and return its URL

https://file.io provides a single-use URL to access uploaded files.
- This service is free.
- The file is deleted once it is downloaded using the URL.

__Note:__ The Vista-3D NIM on NVAIE has a firewall that limits it to receiving input images from AWS and https://file.io.   You cannot upload images from any other service or from your own system.   Once registered, you can download the Vista-3D NIM and use it to process data using local GPU resources and without input image source restrictions.

In [3]:
def post_image_to_fileio(input_image:itk.Image) -> str:
    '''Post an image to file.io and return the link.'''

    # Save the image to a temporary file
    # Note: The Vista-3D NIM on NVAIE is limited to reading .nii.gz files
    tmp_dir = tempfile.mkdtemp()
    tmp_filename = os.path.join(tmp_dir, "tmp.nii.gz")
    itk.imwrite(input_image, tmp_filename)
    
    # Upload the file
    with open(os.path.join(tmp_filename), "rb") as f:
        res = requests.post(
            "https://file.io/",
            files={"file": f}
        )
    if res.status_code != 200:
        raise RuntimeError(f"Cannot upload file. The response {res}.")
    
    # Get the link
    res = res.json()
    link = res["link"]
    
    return link

## Communicate with the Vista-3D NIM on NVAIE

Run the Vista-3D segmentation on given image.

Vista-3D expects images to be 1.5mm isotropic CT scans of large sections of the human body.  Only rudimentary checks are performed to verify these requirements.

In [4]:
def nvaie_vista3d_nim(api_key:str, input_image:itk.Image) -> list:
    '''Run the MONAI VISTA 3D model on the input image and return the result.'''

    # The API endpoint for the MONAI VISTA 3D model on NVIDIA AI Enterprise
    invoke_url = "https://health.api.nvidia.com/v1/medicalimaging/nvidia/vista-3d"
    
    # Check the input image
    assert len(input_image.GetSpacing()) == 3, "The input image must be 3D."
    isotropy = ((input_image.GetSpacing()[1] / input_image.GetSpacing()[0]) +
                (input_image.GetSpacing()[2] / input_image.GetSpacing()[0])) / 2
    if isotropy < 0.9 or isotropy > 1.1 or input_image.GetSpacing()[0] != 1.5:
        print("WARNING: The input image should have 1.5 mm isotropic spacing.  Performance will be degraded.")
        print("    The input image has spacing:", input_image.GetSpacing())
    input_image_arr = itk.array_view_from_image(input_image)
    minv = input_image_arr.min()
    maxv = input_image_arr.max()
    if minv < -1024 or maxv > 3071:
        print("WARNING: The input image should have Hounsfield Units in the range [-1024, 3071].  Performance will be degraded.")
        print("    The input image has Hounsfield Units in the range:", [minv, maxv])

    # Post the image to file.io and get the link
    input_image_url = post_image_to_fileio(input_image)

    # Define the header and payload for the API call
    header = {
    "Authorization": "Bearer " + api_key, 
    }
    
    payload = {
        "image": input_image_url,
        # Optionally limited processing to specific classes
        #"prompts": {
        #    "classes": ["liver", "spleen"]
        #}
    }

    # Call the API
    session = requests.Session()
    response = session.post(invoke_url, headers=header, json=payload)
    
    # Check the response
    response.raise_for_status()
    
    # Get the result
    with tempfile.TemporaryDirectory() as temp_dir:
        z = zipfile.ZipFile(io.BytesIO(response.content))
        z.extractall(temp_dir)
        file_list = os.listdir(temp_dir)
        for filename in file_list:
            filepath = os.path.join(temp_dir, filename)
            if os.path.isfile(filepath) and filename.endswith(".nrrd"):
                # SUCCESS: Return the results
                return itk.imread(filepath, pixel_type=itk.SS)
                    
    # FAILURE: Return None
    return None

## Retrieve NVAIE / NGC API Key

See documentation at the start of this notebook on how to obtain an API key and store it as an environment variable.

In [5]:
ngc_api_key = os.environ['NGC_API_KEY']

## Specify the filename of the image to process

Provide the path to the file via the `input_image_filename` variable, or a default image (\"vista3d-example-1.nii.gz\") will be downloaded and cached in the MONAI_DATA_DIRECTORY (defined by an environment variable) or in a temporary directory.

In [6]:
input_image_filename = None

if input_image_filename == None:
    monai_data_directory = os.environ.get("MONAI_DATA_DIRECTORY")
    if monai_data_directory is not None:
        os.makedirs(monai_data_directory, exist_ok=True)
    else:
        monai_data_directory = tempfile.mkdtemp()
    input_image_filename = os.path.join(
        monai_data_directory,
        "vista3d-example-1.nii.gz"
    )
    if not os.path.exists(input_image_filename):
        resp = requests.get("https://assets.ngc.nvidia.com/products/api-catalog/vista3d/example-1.nii.gz")
        with open(input_image_filename, "wb") as f: 
            f.write(resp.content)

## Process the image and save results


In [7]:
# read the image from disk - a wide variety of formats are supported
input_image = itk.imread(input_image_filename)

# run the model using the api key and input image
output_image = nvaie_vista3d_nim(ngc_api_key, input_image)

# save the results to "output_image.mha"
itk.imwrite(output_image, "./output_image.mha")

    The input image has spacing: itkVectorD3 ([0.902344, 0.902344, 5])


## The rest of this notebook visulizes the results

In [8]:
# Convert the input and output images to PyVista images
image = pv.wrap(itk.vtk_image_from_image(input_image))
labels = pv.wrap(itk.vtk_image_from_image(output_image))

# Extract the contours from the label (output) image
contours = labels.contour_labeled(
    smoothing=True,
    smoothing_num_iterations=10,
    output_mesh_type="triangles",
)

In [9]:
# It is critical to use the "client" mode for PyVista so that vtk.js is used for rendering.
#   vtk.js respects image orientation information, whereas other backends do not.
pv.set_jupyter_backend("client")

In [10]:
# For visualization purposes, we will clip the contours half-way along the x-axis
clipped = contours.clip('x')

# Render the clipped contours and the original image
pl = pv.Plotter()
pl.add_mesh(clipped, cmap="pink", show_scalar_bar=False, opacity=1.0)
pl.add_volume(image, clim=[50,800], cmap="pink", show_scalar_bar=False)
pl.set_background("black")
pl.camera_position = 'xz'
pl.show()

Widget(value='<iframe src="http://localhost:61896/index.html?ui=P_0x21780090a00_0&reconnect=auto" class="pyvis…

In [11]:
# The second visualization will show the image and contours in three orthogonal slices as subplots
pl2 = pv.Plotter(shape=(3, 1))

# The normal directions (dirs) of the slices and the positions (pos) of the camera for each view
dirs = ['x', 'y', 'z']
pos = ['yz', 'xz', 'xy']

# Extract the slices and generate the views for each subplot
for i, dir in enumerate(dirs):
    image_slice = image.slice(normal=dir)
    contours_slice = labels.slice(normal=dir, contour=True)
    pl2.subplot(i,0)
    pl2.add_mesh(image_slice, cmap="bone", clim=[-200,500], show_scalar_bar=False, opacity=1.0)
    pl2.add_mesh(contours_slice, cmap="viridis", show_scalar_bar=False, opacity=1.0, line_width=5)
    pl2.camera_position = pos[i]

pl2.set_background("black")
pl2.show()

Widget(value='<iframe src="http://localhost:61896/index.html?ui=P_0x21786e637c0_1&reconnect=auto" class="pyvis…